<a href="https://colab.research.google.com/github/VandadII/colab-notebooks/blob/main/flux_1_dev_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FLUX.1 Gen


🔧 Install required dependencies and create LoRA folder

In [ ]:
!pip install --upgrade diffusers transformers accelerate torch
!pip install protobuf==3.20.3
!mkdir -p /content/lora  # Create folder for your LoRA .safetensors file

Put your LoRA in the `/content/lora` folder

## Model License
Training currently only works with FLUX.1-dev. Which means anything you train will inherit the non-commercial license. It is also a gated model, so you need to accept the license on HF before using it. Otherwise, this will fail. Here are the required steps to setup a license.

Sign into HF and accept the model access here [black-forest-labs/FLUX.1-dev](https://huggingface.co/black-forest-labs/FLUX.1-dev)

[Get a READ key from huggingface](https://huggingface.co/settings/tokens/new?) and place it in the next cell after running it.

In [ ]:
import getpass
import os

# Prompt for the token
hf_token = getpass.getpass('Enter your HF access token and press enter: ')

# Set the environment variable
os.environ['HF_TOKEN'] = hf_token

print("HF_TOKEN environment variable has been set.")

Enter your HF access token and press enter: ··········
HF_TOKEN environment variable has been set.


## Generate Images with LoRA

In [ ]:
import torch
from diffusers import FluxPipeline
import os

# Enable faster downloads from Hugging Face
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# Create the output folder if it doesn't exist
os.makedirs("/content/generated", exist_ok=True)

# Uncomment the following two lines to use a fixed seed for reproducibility
# seed = 42  # You can change this value as needed
# generator = torch.Generator(device="cuda").manual_seed(seed)

# Load the model
model_id = "black-forest-labs/FLUX.1-dev"
pipeline = FluxPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# Load the LoRA weights
lora_path = "/content/lora/your_lora.safetensors"  # Path to your LoRA file
pipeline.load_lora_weights(lora_path)

# Move the pipeline to GPU
pipeline = pipeline.to("cuda:0")

# Define your prompts (10 empty prompts; replace with your own)
prompts = [
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    '',
    ''
]

negative_prompt = ""  # Empty, as per your original config

# Number of repeats per prompt
num_repeats = 4  # From your original config

# Generation settings from your original config
guidance_scale = 7
num_inference_steps = 100  # Your 'sample_steps'

# Generate images for each non-empty prompt
image_counter = 1
for prompt_idx, prompt in enumerate(prompts):
    if prompt.strip() == "":
        continue  # Skip empty prompts
    for repeat in range(num_repeats):
        print(f"Generating image {image_counter} (Prompt {prompt_idx+1}, Repeat {repeat+1}): {prompt}")
        image = pipeline(
            prompt,
            negative_prompt=negative_prompt,
            height=1024,
            width=1024,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps
            # Uncomment the following line if you enable the fixed seed above
            # generator=generator
        ).images[0]
        output_path = f"/content/generated/ferrah_image_{image_counter}.png"  # No seed in filename since it's random
        image.save(output_path)
        print(f"Image {image_counter} saved to {output_path}")
        image_counter += 1